# Imports

In [1]:
import pandas as pd
import numpy as np

---

# Carga de datos

In [2]:
installs = pd.read_pickle("../../../../data/tp2/installs_tp2_formateado.pkl")

In [3]:
targets_competencia = pd.read_pickle("../../../../data/tp2/ref_hashes_target.pkl")

---

### Columnas utilizadas para generar features

In [4]:
installs["no_atribuida"] = (~installs["attributed"]).astype(np.int8)
installs["attributed"] = installs["attributed"].astype(np.int8)
installs["wifi"] = installs["wifi"].fillna(0).astype(np.int8)

### De installs, me quedo con sólo datos que correspondan a la competencia

In [5]:
installs_competencia = installs.loc[installs["device_id"].isin(targets_competencia.index)]

---

# Cargo features generados hasta el momento

## Sets de entrenamiento

In [6]:
# Ventanas de tres dias
Xs = {}
try:
    print("Cargando features")
    for ventana_nro in range(1,7):    
        Xs[ventana_nro] = pd.read_csv("../../features/ventana_{}_entrenar_installs.csv".format(ventana_nro), index_col=0)
except FileNotFoundError:
    print("No se encontraron los features, se generarán los dataframe requeridos")
    for ventana_nro in range(1,7):    
        Xs[ventana_nro] = installs.loc[installs["ventana_{}".format(ventana_nro)]]["device_id"] \
            .drop_duplicates().to_frame().set_index("device_id").copy()

Cargando features
No se encontraron los features, se generarán los dataframe requeridos


## Set que se usará para predecir

In [7]:
# Features para hacer las predicciones que se suben, corresponden a la ventana 7
Xs_predecir = {}

try:
    print("Cargando features usados para predecir")
    Xs_predecir[7] = pd.read_csv("../../features/predecir_installs.csv", index_col=0)
except FileNotFoundError:
    print("No se encontraron los features usados para predecir, se generará el dataframe requerido")
    Xs_predecir[7] = targets_competencia.copy()

Cargando features usados para predecir
No se encontraron los features usados para predecir, se generará el dataframe requerido


## Diccionario de features

In [8]:
features = {}
# Todos los datasets tienen la misma cantidad y nombre de features, tomo la ventana 1
columnas_ya_generadas = list(Xs[1].columns.values)

---

# Creación de features

## Cantidad de instalaciones dentro de la ventana

In [9]:
def cantidad_de_instalaciones(dataframe, nro_ventana):
    return dataframe.groupby("device_id").agg({"date" : "count"}).rename(columns={"date": "cantidad_instalaciones_installs"})

In [10]:
features["cantidad_de_instalaciones"] = (cantidad_de_instalaciones, ["cantidad_instalaciones_installs"])

---

## Tiempo desde la última instalacion hasta el fin de la ventana

In [11]:
def tiempo_desde_ult_instalacion_hasta_fin_ventana(dataframe, nro_ventana):
    return (np.datetime64("2019-04-2{}".format(nro_ventana)) - dataframe.groupby("device_id").agg({"date" : "max"})["date"]).dt.total_seconds().to_frame().rename(columns={"date": "tiempo_ultima_instalacion_hasta_fin_ventana"})

In [12]:
features["tiempo_desde_ult_instalacion_hasta_fin_ventana"] = (tiempo_desde_ult_instalacion_hasta_fin_ventana, ["tiempo_ultima_instalacion_hasta_fin_ventana"])

---

## Cantidad de aplicaciones distintas instaladas

In [13]:
def cantidad_aplicaciones_distintas_instaladas(dataframe, nro_ventana):
    return dataframe.groupby("device_id").agg({"application_id" : "nunique"}).rename(columns={"application_id": "cantidad_aplicaciones_diferentes_installs"})

In [14]:
features["cantidad_aplicaciones_distintas_instaladas"] = (cantidad_aplicaciones_distintas_instaladas, ["cantidad_aplicaciones_diferentes_installs"])

---

## Cantidad de direcciones ip registradas

In [15]:
def cantidad_direcciones_ip(dataframe, nro_ventana):
    return dataframe.groupby("device_id") \
        .agg({"ip_address" : "nunique"}).rename(columns={"ip_address": "cantidad_ips_diferentes_installs"})

In [16]:
features["cantidad_direcciones_ip"] = (cantidad_direcciones_ip, ["cantidad_ips_diferentes_installs"])

---

## Cantidad instalaciones atribuidas

In [17]:
def cantidad_instalaciones_atribuidas(dataframe, nro_ventana):
    return dataframe.groupby("device_id") \
        .agg({"attributed" : "sum"}).rename(columns={"attributed": "cantidad_instalaciones_atribuidas_installs"})

In [18]:
features["cantidad_instalaciones_atribuidas"] = (cantidad_instalaciones_atribuidas, ["cantidad_instalaciones_atribuidas_installs"])

---

## Cantidad instalaciones no atribuidas

In [19]:
def cantidad_instalaciones_no_atribuidas(dataframe, nro_ventana):
    return dataframe.groupby("device_id") \
        .agg({"no_atribuida" : "sum"}).rename(columns={"no_atribuida": "cantidad_instalaciones_no_atribuidas_installs"})

In [20]:
features["cantidad_instalaciones_no_atribuidas"] = (cantidad_instalaciones_no_atribuidas, ["cantidad_instalaciones_no_atribuidas_installs"])

---

## Cantidad de idiomas registrados

In [21]:
def cantidad_idiomas(dataframe, nro_ventana):
    return dataframe.groupby("device_id") \
        .agg({"device_language" : "count"}).rename(columns={"device_language": "cantidad_idiomas_installs"})

In [22]:
features["cantidad_idiomas"] = (cantidad_idiomas, ["cantidad_idiomas_installs"])

---

## Cantidad de instalaciones con wifi

In [23]:
def cantidad_instalaciones_con_wifi(dataframe, nro_ventana):
    return dataframe.groupby("device_id") \
        .agg({"wifi" : "sum"}).rename(columns={"wifi": "cantidad_instalaciones_wifi_installs"})

In [24]:
features["cantidad_instalaciones_con_wifi"] = (cantidad_instalaciones_con_wifi, ["cantidad_instalaciones_wifi_installs"])

---

## Instalo una de las top-K aplicaciones

In [50]:
K = 10

def instalo_una_aplicacion_top(dataframe, nro_ventana):
    features = pd.DataFrame()
    for ranking in range(1, K+1):
        top_k = installs["application_id"].value_counts().iloc[:K].index
        features = features.merge(dataframe.groupby("device_id")\
            .agg({"application_id" : lambda x: x.isin(top_k).any()}), how="right", right_index=True, left_index=True)\
            .rename(columns={"application_id" : "instalo_aplicacion_top_{}".format(ranking)})
        features["instalo_aplicacion_top_{}".format(ranking)] = features["instalo_aplicacion_top_{}".format(ranking)].astype(np.int8)
    return features

In [52]:
features["instalo_una_aplicacion_top"] = (instalo_una_aplicacion_top, ["instalo_aplicacion_top_{}".format(x) for x in range(1, K+1)])

---

## Tiene uno de los top-K idiomas registrados

In [68]:
K = 10

def tiene_uno_de_los_top_idiomas(dataframe, nro_ventana):
    features = pd.DataFrame()
    for ranking in range(1, K+1):
        top_k = installs["device_language"].value_counts().iloc[:K].index
        features = features.merge(dataframe.groupby("device_id")\
            .agg({"device_language" : lambda x: x.isin(top_k).any()}), how="right", right_index=True, left_index=True)\
            .rename(columns={"device_language" : "tiene_idioma_top_{}".format(ranking)})
        features["tiene_idioma_top_{}".format(ranking)] = features["tiene_idioma_top_{}".format(ranking)].astype(np.int8)
    return features

In [69]:
features["tiene_uno_de_los_top_idiomas"] = (tiene_uno_de_los_top_idiomas, ["tiene_idioma_top_{}_installs".format(x) for x in range(1, K+1)])

---

# Genero los features

## Función generadora de features en ventanas

In [54]:
def generar_feature_en_ventanas(dataframe, generador_feature, destinos, ventana_inicial, ventana_final):
    """El rango se toma como python, [ventana_inicial, ventana_final)"""
    for ventana_nro in range(ventana_inicial, ventana_final):
        feature = generador_feature(dataframe.loc[dataframe["ventana_{}".format(ventana_nro)]], ventana_nro)
        destinos[ventana_nro] = destinos[ventana_nro].merge(feature, left_index=True, right_index=True, how="left")

### Genero los features

In [70]:
COLUMNAS_QUE_GENERA_EL_FEATURE = 1
FUNCION_PARA_CALCULAR_EL_FEATURE = 0

for nombre_feature in features:    
    columnas_generadas = features[nombre_feature][COLUMNAS_QUE_GENERA_EL_FEATURE]
    feature_ya_generado = False
    for columna in columnas_generadas:
        if columna in columnas_ya_generadas:
            feature_ya_generado = True
            break
    
    if feature_ya_generado:
        continue
    
    print("Generando feature {} en ventanas".format(nombre_feature))
    generar_feature_en_ventanas(installs, features[nombre_feature][FUNCION_PARA_CALCULAR_EL_FEATURE], Xs, 1, 7)
    print("Generando feature {} en set de predicciones".format(nombre_feature))
    generar_feature_en_ventanas(installs_competencia, features[nombre_feature][FUNCION_PARA_CALCULAR_EL_FEATURE], Xs_predecir, 7, 8)
    columnas_ya_generadas.extend(features[nombre_feature][COLUMNAS_QUE_GENERA_EL_FEATURE])

Generando feature tiene_uno_de_los_top_idiomas en ventanas
Generando feature tiene_uno_de_los_top_idiomas en set de predicciones


---

# Guardado de los sets de entrenamiento y el set de prediccion

In [71]:
for ventana_nro in range(1, 7):
    Xs[ventana_nro].to_csv("../../features/ventana_{}_entrenar_installs.csv".format(ventana_nro))

Xs_predecir[7].to_csv("../../features/predecir_installs.csv")

---